In [18]:
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from d2l import mindspore as d2l

In [2]:
def trans_conv(X, K):
    h, w = K.shape
    Y = ops.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

In [3]:
X = mindspore.Tensor([[0.0, 1.0], [2.0, 3.0]])
K = mindspore.Tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

Tensor(shape=[3, 3], dtype=Float32, value=
[[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
 [ 0.00000000e+00,  4.00000000e+00,  6.00000000e+00],
 [ 4.00000000e+00,  1.20000000e+01,  9.00000000e+00]])

In [7]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.Conv2dTranspose(1, 1, kernel_size=2, pad_mod='valid', has_bias=False, weight_init=K)
tconv(X)

Tensor(shape=[1, 1, 3, 3], dtype=Float32, value=
[[[[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
   [ 0.00000000e+00,  4.00000000e+00,  6.00000000e+00],
   [ 4.00000000e+00,  1.20000000e+01,  9.00000000e+00]]]])

In [5]:
tconv = nn.Conv2dTranspose(1, 1, kernel_size=2, pad_mode='pad', 
                           padding=1, has_bias=False, weight_init=K)
tconv(X)

Tensor(shape=[1, 1, 1, 1], dtype=Float32, value=
[[[[ 4.00000000e+00]]]])

In [9]:
tconv = nn.Conv2dTranspose(1, 1, kernel_size=2, stride=2, pad_mode='valid', has_bias=False, weight_init=K)
tconv(X)

Tensor(shape=[1, 1, 4, 4], dtype=Float32, value=
[[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
   [ 0.00000000e+00,  0.00000000e+00,  2.00000000e+00,  3.00000000e+00],
   [ 0.00000000e+00,  2.00000000e+00,  0.00000000e+00,  3.00000000e+00],
   [ 4.00000000e+00,  6.00000000e+00,  6.00000000e+00,  9.00000000e+00]]]])

In [16]:
import mindspore.common.initializer as init

X = ops.randn((1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, pad_mode='pad', padding=2, stride=3, has_bias=True)
k = (conv.group / (conv.in_channels * conv.kernel_size[0] * conv.kernel_size[1])) ** 0.5
conv.weight_init = init.initializer(init.Uniform(k), conv.weight.shape, conv.weight.dtype)
conv.bias_init = init.initializer(init.Uniform(k), conv.bias.shape, conv.bias.dtype)

tconv = nn.Conv2dTranspose(20, 10, kernel_size=5, pad_mode='pad', padding=2, stride=3)
conv.weight_init = init.initializer(init.Uniform(k), conv.weight.shape, conv.weight.dtype)
conv.bias_init = init.initializer(init.Uniform(k), conv.bias.shape, conv.bias.dtype)
tconv(conv(X)).shape == X.shape

True

In [21]:
X = ops.arange(9.0).reshape(3, 3)
K = mindspore.Tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

Tensor(shape=[2, 2], dtype=Float32, value=
[[ 2.70000000e+01,  3.70000000e+01],
 [ 5.70000000e+01,  6.70000000e+01]])

In [20]:
def kernel2matrix(K):
    k, W = ops.zeros(5), ops.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

Tensor(shape=[4, 9], dtype=Float32, value=
[[ 1.00000000e+00,  2.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00,  1.00000000e+00,  2.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  3.00000000e+00,  4.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  3.00000000e+00,  4.00000000e+00]])

In [22]:
Y == ops.matmul(W, X.reshape(-1)).reshape(2, 2)

Tensor(shape=[2, 2], dtype=Bool, value=
[[ True,  True],
 [ True,  True]])

In [23]:
Z = trans_conv(Y, K)
Z == ops.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

Tensor(shape=[3, 3], dtype=Bool, value=
[[ True,  True,  True],
 [ True,  True,  True],
 [ True,  True,  True]])